In [1]:
import socket
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import scipy.signal as sig
import datetime

* AF_INET -> relates to the type of address that socket can interact with, in this case its IPv4
* SOCK_STREAM -> relates to TCP (connection-based portocol)
* SOCK_DGRAM -> relates to UDP (datagram-based protocol), send a msg, get reply, end connection
* Should use TCP for this
* SOL_SOCKET & SO_REUSEADDR -> allows the addr/port to be reused immediately instead of being stuck in TIME_WAIT waiting for late packets

In [2]:
def startServer(hostName = socket.gethostbyname('0.0.0.0'), portNum = 6000, bufferSize = 2048):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind((hostName, portNum))
    sock.settimeout(100)
    sock.listen(1)
    print('Listening on {}'.format(portNum))
    return sock

In [3]:
def connectToRasp(socketObj):
    conn, addr = socketObj.accept()
    print("Connection address: ", addr)
    return conn

In [25]:
def getDataFromRasp(connectionObj, decodeType='str', bufferSize = 2048):
    # default is np array b/c for the most part the computer will be getting images
    if decodeType == 'str':
        return connectionObj.recv(bufferSize).decode('utf-8')
    if decodeType == 'list':
        return connectionObj.recv(bufferSize)

In [5]:
def sendDataToRasp(connectionObj, data):
    connectionObj.send(bytes(data, 'utf-8'))

In [6]:
def close(socketObj, connectionObj):
    socketObj.close()
    connectionObj.close()

In [96]:
s = startServer()
raspC = connectToRasp(s)

Listening on 6000
Connection address:  ('192.168.29.171', 48824)


#### Comm Protocol
1. Server sends "Start" -> to signal that it is okay to receive image data
2. Client send Image data -> waits for Server to send "Start"
3. Server collects all data that Client sent
4. Once collection is done -> send "Start" to Client
5. Server sends "Quit" -> End connection

In [102]:
def requestImageFromClient(connectionObj):
    # Init the process
    sendDataToRasp(connectionObj, 'send image')
    # Agreeed upon resolution, 640 by 480, will be stored below
    skele = np.zeros((640, 480,3), dtype=np.uint8)
    
    for i in range(640):
        tempRow = list(getDataFromRasp(connectionObj, decodeType='list'))
        if len(tempRow) != 480:
            print('not 480')
            break
        skele[i, :, 2] = tempRow
        sendDataToRasp(connectionObj, 'next row')
        
    for i in range(640):
        tempRow = list(getDataFromRasp(connectionObj, decodeType='list'))
        if len(tempRow) != 480:
            print('not 480')
            break
        skele[i, :, 1] = tempRow
        sendDataToRasp(connectionObj, 'next row')
        
    for i in range(640):
        tempRow = list(getDataFromRasp(connectionObj, decodeType='list'))
        if len(tempRow) != 480:
            print('not 480')
            break
        skele[i, :, 0] = tempRow
        sendDataToRasp(connectionObj, 'next row')
    
    return skele

In [100]:
startTime = datetime.datetime.now()

In [101]:
for i in range(50):
    imgD = requestImageFromClient(raspC)
    Image.fromarray(imgD).save('images/imgD{}.jpg'.format(i))
    print(i)
endTime = datetime.datetime.now()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [95]:
close(s, raspC)

In [103]:
import cv2

In [104]:
sendDataToRasp(raspC, 'end')

In [105]:
close(s, raspC)